# Pipeline Task API

<br>Owner: **Alex Drlica-Wagner** ([@kadrlica](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@kadrlica))
<br>Last Verified to Run: **2018-08-10**
<br>Verified Stack Release: **v16.0**

## Learning Objectives:

This notebook seeks to teach users how to unpack a pipeline tasks. As an example, we focus on `processCcd.py`, with the goal of diving into the configuration, interface, and structure of pipeline tasks. This notebook is a digression from Justin Myles script that demonstrates how to run a series of pipeline tasks from the command line to rerun HSC data processing [link].

After working through this tutorial you should be able to:

* Find the source code for a pipeline task
* Configure (and investigate the configuration) of pipeline tasks
* Investigate and run those tasks in python

## Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

In [ ]:
# What version of the Stack are we using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
# Filter some warnings printed by v16.0 of the stack
import warnings
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)

In [ ]:
%matplotlib inline
import os
import pydoc
import matplotlib.pyplot as plt

In [ ]:
import lsst.afw.display     as afwDisplay

## Diving into a Pipeline Task

Our goal is to dive into the inner workings of `ProcessCcd.py`. We pickup from the command line processing described in the getting started tutorials [here](https://pipelines.lsst.io/getting-started/data-setup.html#) and [here](https://pipelines.lsst.io/getting-started/processccd.html), as well as Justin Myles HSC reprocessing notebook [here](). We are specifically interested in digging into the following line:

```
processCcd.py $DATADIR --rerun processCcdOutputs --id
```

We start by tracking down the location of the `processCcd.py` shell script

In [ ]:
!(which processCcd.py)

This is the proverbial "end of the thread". Our goal is to pull on this thread to unravel the python/C++ functions that are being called under the hood. We start by taking a peak in this script

In [ ]:
!cat $(which processCcd.py)

Ok, this hasn't gotten us very far, but after getting through the stock header, we now have the next link in our chain:
```
from lsst.pipe.tasks.processCcd import ProcessCcdTask
```

There are two ways we can proceed from here. One is to [Google](http://lmgtfy.com/?q=lsst.pipe.tasks.processCcd) `lsst.pipe.tasks.processCcd`, which will take us to this [doxygen page](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/classlsst_1_1pipe_1_1tasks_1_1process_ccd_1_1_process_ccd_task.html) and/or the soure code on [GitHub](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/processCcd.py). 

The second approach is to do the import the class oursleves and try to investigate it interactively.


In [ ]:
import lsst.pipe.tasks.processCcd
from lsst.pipe.tasks.processCcd import ProcessCcdTask, ProcessCcdConfig

We can get to the source code for these classes directly using the [`stackclub` toolkit module](https://stackclub.readthedocs.io/), as shown in the [FindingDocs.ipynb](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/GettingStarted/FindingDocs.ipynb)

In [ ]:
from stackclub import where_is
where_is(ProcessCcdConfig)

# Diving into a Task Config

Pipeline tasks are controlled and tweaked through there associated `TaskConfig` objects. To investigate the configuration parameters of the `ProcessCcdTask`, we create an instance of the `ProcessCcdConfig` and try calling the `help` method (commented out for brevity). What we are really interested in are the "Data descriptors", which we can print directly after capturing the documentation output by `help`.

The first step is to try to get at the documentation through the `help` function (commented out below for brevity). We can find 

The `ProcessCcdConfig` object contains of both raw configurables like `doCalibrate` and other configs, like `isr`. To investigate one of these in more detail, we can import it and query it's "Data descriptors".


In [ ]:
from lsst.ip.isr.isrTask import IsrTask, IsrTaskConfig
print('\n'.join(pydoc.render_doc(IsrTaskConfig).split('\n')[16:40]) + '...')

These configurationas are pretty self-explanitory, but say that we really want to understand what `doFringe` is doing. Inorder to get that information we need to go to the source code.

In [ ]:
where_is(IsrTaskConfig)

We can then search this file for `doFringe` and we find [several lines](https://github.com/lsst/ip_isr/blob/cc4efb7d763d3663c9e989339505df9654f23fd9/python/lsst/ip/isr/isrTask.py#L597-L598) that look like this:

        if self.config.doFringe and not self.config.fringeAfterFlat:
            self.fringe.run(ccdExposure, **fringes.getDict())
            
If we want to go deeper to see what `fringe.run` does, we can repeat the above process

In [ ]:
isr_task = IsrTask()
print(isr_task.fringe)
import  lsst.ip.isr.fringe
where_is(lsst.ip.isr.fringe.FringeTask)

We finally make our way to the source code for [FringeTask.run](https://github.com/lsst/ip_isr/blob/cc4efb7d763d3663c9e989339505df9654f23fd9/python/lsst/ip/isr/fringe.py#L104), which gives us details on how the fringe correction is performed (i.e. by creating a fringe image and subtracting it from the data image).

# Running a Task

Now that we've figured out how to investigate the config for a task, let's try to run the task itself.

In [ ]:
# First we setup the directories for the raw image and calibration.
import shutil
basedir = '/project/stack-club/validation_data_hsc'
repodir = os.path.join(basedir,'data')
# By default the calibration directory lives in ${datadir}/CALIB; 
# however, in the validation_data_hsc it lives in ${basedir}/CALIB
calibdir = os.path.join(basedir,'CALIB')
# The directory for our processed output
outdir = '/home/kadrlica/tmpdir/'

In [ ]:
# Next, we create a butler to manage the data repositories
from lsst.daf.persistence import Butler
# output directory cannot exist (wait for Gen3 Butler...)
if os.path.exists(outdir): shutil.rmtree(outdir)

# This strange structure for Butler initialization is a 'feature' of the Gen2 butler 
# and should be replaced in Gen3
butler = Butler(inputs={'root': repodir, 'mapperArgs': {'calibRoot': calibdir}}, 
                outputs={'root': outdir})

We can use the butler to list the available data sets, and to choose the CCD that we want to process.

In [ ]:
# We can list available data sets
ccds = butler.queryMetadata('raw',['visit','ccd'])
print(ccds[0:10],'...')

In [ ]:
# We choose one of these CCDs as our dataRef
dataRef = butler.dataRef('raw', dataId={'visit':903332,'ccd':25})

Now we are prepared to create our `IsrTaskConfig`. At this point, we will turn off many of the processing steps to speed up the task.

In [ ]:
config = IsrTaskConfig()
config.doBias = True
config.doDark = True
config.doFlat = True

With the `config` created and configured, we can now create the `IsrTask`

In [ ]:
isr_task = IsrTask(config)

And run it!

In [ ]:
%%time
struct = isr_task.runDataRef(dataRef)

The output is a `struct` containing the processed exposure:

In [ ]:
print(struct)

In [ ]:
afwDisplay.setDefaultBackend('matplotlib') 
# ADW: why can't we set the backend before this cell?

figure = plt.figure(1,figsize=(10,10))
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(dataRef.get())
plt.title("Raw")

figure = plt.figure(2,figsize=(10,10))
afw_display = afwDisplay.Display(2)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(struct.exposure)
plt.title("ISR 'Corrected'")

In [ ]:
for maskName, maskBit in struct.exposure.mask.getMaskPlaneDict().items():
    print('{}: {}'.format(afw_display.getMaskPlaneColor(maskName),maskName))

In [ ]:
# https://github.com/ipython/ipykernel/issues/111#issuecomment-237089618
# Try to get the task outputs into the notebook (and failing)
import sys,logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# ch.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]

## Run processCcdTask

First, we run the task as we would from the command line. All that we are doing here is parsing the `cmdline` string as if it is arguments passed to `processCcdTask.py` on the command line. Unfortunately, the task does not print it's output to a notebook cell, so we just need to wait for ~1 minute for this to run. We turn off several optional subtask steps so that things run faster...

In [ ]:
# Create the task
task = ProcessCcdTask()

In [ ]:
%%timeit
# output directory cannot exist (wait for Gen3 Butler...)
if os.path.exists(outdir): shutil.rmtree(outdir)
cmdline = '{0} --calib {1} --output {2} --id ccd=25 visit=903982'.format(repodir,calibdir,outdir)
cmdline += ' --config doCalibrate=False isr.doBias=False isr.doDark=False isr.doFlat=False'
struct = task.parseAndRun(cmdline.split())

In [ ]:
config = ProcessCcdConfig()
#help(config)
helplist = pydoc.render_doc(config).split('\n')
print('\n'.join(helplist[18:47]))

The next level in complexity is to try to call `task.run`. In order to do this we need to provide a `dataRef`. To do this, we create a butler.

In [ ]:
# Since we don't want to do calibration processing we turn off these configs
config = ProcessCcdConfig()
config.isr.doBias = False
config.isr.doDark = False
config.isr.doFlat = False
config.doCalibrate = False

In [ ]:
task = ProcessCcdTask(butler=butler, config=config)

In [ ]:
# Setup the astrometry reference catalogs following:
# https://github.com/lsst/validation_data_hsc
#!export SETUP_ASTROMETRY_NET_DATA="astrometry_net_data sdss-dr9-fink-v5b"
#!export ASTROMETRY_NET_DATA_DIR={repodir + '/sdss-dr9-fink-v5b'}

# TODO: Create the astrometry reference object to pass to the task